# Intro to AI - Coursework
Joshua Luke Boddy - aczc760

Saeed Almansoori - acwk457

### To-Do
- Functions to prepare each column of the dataset we need
- Decide on learning model (Multivariate regression?)
- Research integer value return (Possible with Regression)

## Data Preparation

Taking a CSV file as our data set, the data needs to be converted across from CSV to something program readable as well as formatting the data into a readable format for our model (undecided).

In [ ]:
import pandas as pd
import numpy as np
import scikit-learn as sk
import matplotlib.pyplot as plt

In [ ]:
def prepare_categorical_column(value, key):
    for k, v in conversion_keys.items(): # Traverse the dictionary 
        if value in conversion_keys:  #  prints if value exists and breaks the loop
            print ("Value exists")  
            break
        else:
            conversion_keys[key] = value  # If value does not exist add it to the dictionary
            print (key, value)          # Return newly added key and value
    

In [ ]:
cars_data = pd.read_csv('./cars.csv')
conversion_keys = {}

In [ ]:
def prepare_dataset():
    # Iterate over the columns inside the DataFrame
    # For each column, run prepare_categorical_column as a lambda

## Model Implementation
TBD when we confirm whether we can use multivariate regression or not